
<span style="float:right"><a href="http://moldesign.bionano.autodesk.com/" target="_blank" title="About">About</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="https://forum.bionano.autodesk.com/c/Molecular-Design-Toolkit" target="_blank" title="Forum">Forum</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="https://github.com/autodesk/molecular-design-toolkit/issues" target="_blank" title="Issues">Issues</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="http://bionano.autodesk.com/MolecularDesignToolkit/explore.html" target="_blank" title="Tutorials">Tutorials</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="http://autodesk.github.io/molecular-design-toolkit/" target="_blank" title="Documentation">Documentation</a></span>
</span>
![Molecular Design Toolkit](img/top.png)
<br>

<center><h1>Example 3: Simulating HIV Protease </h1> </center>

This notebook prepares a co-crystallized protein / small molecule ligand structure from [the PDB database](http://www.rcsb.org/pdb/home/home.do) and prepares it for molecular dynamics simulation. 

---

In [ ]:
import moldesign as mdt
import moldesign.units as u


## I. The crystal structure

First, we'll download and investigate the [3AID crystal structure](http://www.rcsb.org/pdb/explore.do?structureId=3aid).

### A. Download and visualize

In [ ]:
mol = mdt.read('../data/3AID.pdb')
mol

### B. Try assigning a forcefield

This structure is not ready for MD - this command will raise a `ParameterizationError` Exception. After running this calculation, click on the **Errors/Warnings** tab to see why.

In [ ]:
newmol = mdt.assign_forcefield(mol)

You should see 3 errors: 
 * The residue name `ARQ` not recognized
 * Atom `HD1` in residue `HIS69`, chain `A` was not recognized
 * Atom `HD1` in residue `HIS69`, chain `B` was not recognized
 
(There's also a warning about bond distances, but these can be generally be fixed with an energy minimization before running dynamics)

We'll deal with the histidine residues first.

## II. Prepping the protein

### A. Assign histidine's protonation
Histidine is notoriously tricky, because it exists in no less than three different protonation states at biological pH (7.4) - the "delta-protonated" form, referred to with residue name `HID`; the "epsilon-protonated" form aka `HIE`; and the doubly-protonated form `HIP`, which has a +1 charge. These are drawn below.

In [ ]:
hid = mdt.from_smiles('O=C(O)[C@@H](N)Cc1c[nH]cn1')
hid.draw3d()

In [ ]:
hie = mdt.from_smiles('O=C(O)[C@@H](N)Cc1cnc[nH]1')
hie.draw3d()

Crystallographers usually can't resolve hydrogen positions, and so can't tell you which form of histidine you have. PDB files thus refer to histidine as `HIS`, which leaves the protonation state ambiguous. In general, picking the right protontation is both extremely important and extremely difficult.

We're in luck for this simulation, however - the histidines are located on the surface of the protein, quite far from the drug binding site, and so their state is not likely to be important for a drug binding calculation. Further, the presence of the `HD1` atoms in the crystal structure indicates that we should probably go ahead and change our `HIS` residues to `HID`.

The fix is easy - just change the name. We'll rerun the paramterization to confirm that the histidine errors disappeared (although the small molecule errors remain).

In [ ]:
for residue in mol.residues:
    if residue.resname == 'HIS':
        residue.resname = 'HID'
        print 'I changed %s in chain %s to HID' % (residue.name, residue.chain.name)

In [ ]:
md_ready_molecule = mdt.assign_forcefield(mol)

### B. Remove the ligand
The following cell creates a new ``Molecule`` with only the protein - no solvent or bound ligand.

In [ ]:
receptor_structure = mdt.Molecule([atom for atom in mol.atoms if atom.residue.type == 'protein'])
receptor = mdt.assign_forcefield(receptor_structure)

### C. Prep for dynamics

In [ ]:
receptor.set_energy_model(mdt.models.OpenMMPotential, implicit_solvent='obc', cutoff=8.0*u.angstrom)
receptor.set_integrator(mdt.integrators.OpenMMLangevin, timestep=2.0*u.fs)

In [ ]:
receptor.configure_methods()

### D. Equilibrate the protein
The next series of cells first minimize the crystal structure to remove clashes, then heats the system to 300K.

In [ ]:
mintraj = receptor.minimize()
mintraj.draw()

In [ ]:
# Restrain the peptide backbone:
for residue in receptor.residues:
    for atom in residue.backbone:
        receptor.constrain_atom(atom)
print 'Constrained %d atoms' % len(receptor.constraints)

In [ ]:
warmup = receptor.run(20.0*u.ps)

In [ ]:
receptor.clear_constraints()
receptor.integrator.params.frame_interval=0.1*u.ps
equil = receptor.run(5.0*u.ps)

In [ ]:
fulltraj = mintraj + warmup + equil
fulltraj.draw()

## III. Parameterizing a small molecule
We'll use the GAFF (generalized Amber force field) to create force field parameters for the small ligand.

### A. Isolate the small molecule
Click on the ligand to select it, then we'll use that selection to create a new molecule.

In [ ]:
sel = mdt.widgets.ResidueSelector(mol)
sel

In [ ]:
drugres = mdt.Molecule(sel.selected_residues[0])
drugres.draw2d()

### B. Assign bond orders and hydrogens
The crystal structure doesn't provide indicate bond orders or hydrogens, but we can add those:

In [ ]:
drugmol = mdt.add_hydrogen(mdt.guess_bond_orders((drugres)))
drugmol.draw2d()

In [ ]:
drugmol.draw()

### C. Assign partial charges and a GAFF forcefield

To assign force field parameters to this molecule, each atom will need 1) a GAFF atom type, and 2) a partial charge.

For the charges, we'll use the fast (but not particularly accurate) AM1-BCC method to compute partial charges for the atoms.

In [ ]:
assert drugmol.charge == +1

In [ ]:
result = drugmol.get_gaff_parameters(charge='am1-bcc', )

In [ ]:
print drugmol.write(format='pdb')

In [ ]:
mol = bb.from_smiles('C1=CCNOC1')

In [ ]:
#include commands as file ... stdout, stderr in main display

In [ ]:
reload(bb.interfaces.ambertools)
bb.interfaces.ambertools.am1_bcc_charges(mol)

In [ ]:
mol = mdt.from_name('bipyridine')

In [ ]:
mol.draw(height=300, width=300, show_2dhydrogens=True)

In [ ]:
mdt.widgets.Symmetrizer(mdt.from_smiles('CC'))